## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,7.232,94,100,6.63,overcast clouds
1,1,Porto Velho,BR,-8.7619,-63.9039,80.654,83,75,2.06,broken clouds
2,2,Cap-Aux-Meules,CA,47.3821,-61.8597,41.054,60,20,5.66,few clouds
3,3,Channel-Port Aux Basques,CA,47.5702,-59.1367,34.862,71,8,10.27,clear sky
4,4,Itarema,BR,-2.9248,-39.9167,84.632,70,32,5.52,scattered clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
  # Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)                                                                                            

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Porto Velho,BR,-8.7619,-63.9039,80.654,83,75,2.06,broken clouds
4,4,Itarema,BR,-2.9248,-39.9167,84.632,70,32,5.52,scattered clouds
5,5,Kikwit,CD,-5.0410,18.8162,86.990,54,69,0.68,broken clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,84.650,65,9,3.23,clear sky
14,14,Bambous Virieux,MU,-20.3428,57.7575,79.052,78,75,3.09,broken clouds
15,15,Adrar,MR,20.5022,-10.0711,89.600,10,74,6.97,broken clouds
20,20,Georgetown,MY,5.4112,100.3354,85.946,83,20,2.06,few clouds
21,21,Port Alfred,ZA,-33.5906,26.8910,83.174,44,78,1.79,broken clouds
22,22,Souillac,MU,-20.5167,57.5167,79.178,78,75,3.09,broken clouds
27,27,Jalu,LY,29.0331,21.5482,86.702,11,75,5.42,broken clouds


In [12]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count() 

233


City_ID                233
City                   233
Country                224
Lat                    233
Lng                    233
Max Temp               233
Humidity               233
Cloudiness             233
Wind Speed             233
Current Description    233
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

224


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Porto Velho,BR,-8.7619,-63.9039,80.654,83,75,2.06,broken clouds
4,4,Itarema,BR,-2.9248,-39.9167,84.632,70,32,5.52,scattered clouds
5,5,Kikwit,CD,-5.0410,18.8162,86.990,54,69,0.68,broken clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,84.650,65,9,3.23,clear sky
14,14,Bambous Virieux,MU,-20.3428,57.7575,79.052,78,75,3.09,broken clouds


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Porto Velho,BR,80.654,broken clouds,-8.7619,-63.9039,
4,Itarema,BR,84.632,scattered clouds,-2.9248,-39.9167,
5,Kikwit,CD,86.990,broken clouds,-5.0410,18.8162,
7,Hithadhoo,MV,84.650,clear sky,-0.6000,73.0833,
14,Bambous Virieux,MU,79.052,broken clouds,-20.3428,57.7575,
15,Adrar,MR,89.600,broken clouds,20.5022,-10.0711,
20,Georgetown,MY,85.946,few clouds,5.4112,100.3354,
21,Port Alfred,ZA,83.174,broken clouds,-33.5906,26.8910,
22,Souillac,MU,79.178,broken clouds,-20.5167,57.5167,
27,Jalu,LY,86.702,broken clouds,29.0331,21.5482,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Porto Velho,BR,80.654,broken clouds,-8.7619,-63.9039,Ecos Hotel Conforto
4,Itarema,BR,84.632,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
5,Kikwit,CD,86.990,broken clouds,-5.0410,18.8162,HOTEL PLACE YALABA
7,Hithadhoo,MV,84.650,clear sky,-0.6000,73.0833,Scoop Guest House
14,Bambous Virieux,MU,79.052,broken clouds,-20.3428,57.7575,Casa Tia Villa
15,Adrar,MR,89.600,broken clouds,20.5022,-10.0711,
20,Georgetown,MY,85.946,few clouds,5.4112,100.3354,Cititel Penang
21,Port Alfred,ZA,83.174,broken clouds,-33.5906,26.8910,The Halyards Hotel
22,Souillac,MU,79.178,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
27,Jalu,LY,86.702,broken clouds,29.0331,21.5482,صاحبة الفخامة


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Deliverable 2 End